In [48]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import gmaps.datasets
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [49]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,city,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,zhangye,CN,38.9342,100.4517,291.60,61.0,82.0,1.18,broken clouds
1,1,kapaa,US,22.0752,-159.3190,303.33,78.0,1.0,1.79,clear sky
2,2,upington,ZA,-28.4478,21.2561,287.55,20.0,0.0,4.12,clear sky
3,3,mataura,NZ,-46.1927,168.8643,281.19,95.0,100.0,1.02,overcast clouds
4,4,kaeo,NZ,-35.1000,173.7833,287.22,74.0,32.0,8.88,light rain


In [51]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 280
What is the maximum temperature you would like for your trip? 290


In [52]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_city_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &  (city_data_df["Max Temp"] <= max_temp)]
preferred_city_df

,City_ID,city,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,upington,ZA,-28.4478,21.2561,287.55,20.0,0.0,4.12,clear sky
3,3,mataura,NZ,-46.1927,168.8643,281.19,95.0,100.0,1.02,overcast clouds
4,4,kaeo,NZ,-35.1000,173.7833,287.22,74.0,32.0,8.88,light rain
8,8,barrow,US,71.2906,-156.7887,280.16,81.0,90.0,9.26,overcast clouds
9,9,bratsk,RU,56.1325,101.6142,286.42,91.0,47.0,2.44,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
754,754,general roca,AR,-39.0333,-67.5833,287.37,25.0,55.0,4.13,broken clouds
757,757,berlevag,NO,70.8578,29.0864,281.51,82.0,7.0,1.36,clear sky
763,763,okaihau,NZ,-35.3167,173.7833,286.21,75.0,43.0,6.02,scattered clouds
765,765,mezen,RU,65.8522,44.2400,286.20,97.0,52.0,2.23,broken clouds


In [53]:
# 4a. Determine if there are any empty rows.
preferred_city_df.count()

City_ID                189
city                   189
Country                188
Lat                    189
Lng                    189
Max Temp               189
Humidity               189
Cloudiness             189
Wind Speed             189
Current Description    189
dtype: int64

In [54]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_city_df.dropna()

In [55]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["city", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,city,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,upington,ZA,287.55,clear sky,-28.4478,21.2561,
3,mataura,NZ,281.19,overcast clouds,-46.1927,168.8643,
4,kaeo,NZ,287.22,light rain,-35.1000,173.7833,
8,barrow,US,280.16,overcast clouds,71.2906,-156.7887,
9,bratsk,RU,286.42,scattered clouds,56.1325,101.6142,
17,ancud,CL,283.57,overcast clouds,-41.8697,-73.8203,
20,carnarvon,AU,280.19,clear sky,-24.8667,113.6333,
26,chuy,UY,283.02,few clouds,-33.6971,-53.4616,
30,cidreira,BR,285.13,clear sky,-30.1811,-50.2056,
35,lasa,IT,287.05,scattered clouds,46.6166,10.7002,


In [56]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lng = row["Lng"]
    lat = row["Lat"]
    
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotel_data =  requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
    except (KeyError, IndexError):
        print("Hotel not found... skipping.")
        
hotel_df

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,city,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,upington,ZA,287.55,clear sky,-28.4478,21.2561,River Place Manor
3,mataura,NZ,281.19,overcast clouds,-46.1927,168.8643,Ellie's Villa
4,kaeo,NZ,287.22,light rain,-35.1000,173.7833,Bushido Lodge
8,barrow,US,280.16,overcast clouds,71.2906,-156.7887,King Eider Inn
9,bratsk,RU,286.42,scattered clouds,56.1325,101.6142,"Gostinitsa ""Shvedka"""
...,...,...,...,...,...,...,...
754,general roca,AR,287.37,broken clouds,-39.0333,-67.5833,Hotel y Casino del Río General Roca
757,berlevag,NO,281.51,clear sky,70.8578,29.0864,Kvitbrakka
763,okaihau,NZ,286.21,scattered clouds,-35.3167,173.7833,Macadamia Farm
765,mezen,RU,286.20,broken clouds,65.8522,44.2400,Gostevoy Dom Everest


In [57]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,city,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,upington,ZA,287.55,clear sky,-28.4478,21.2561,River Place Manor
3,mataura,NZ,281.19,overcast clouds,-46.1927,168.8643,Ellie's Villa
4,kaeo,NZ,287.22,light rain,-35.1000,173.7833,Bushido Lodge
8,barrow,US,280.16,overcast clouds,71.2906,-156.7887,King Eider Inn
9,bratsk,RU,286.42,scattered clouds,56.1325,101.6142,"Gostinitsa ""Shvedka"""
...,...,...,...,...,...,...,...
754,general roca,AR,287.37,broken clouds,-39.0333,-67.5833,Hotel y Casino del Río General Roca
757,berlevag,NO,281.51,clear sky,70.8578,29.0864,Kvitbrakka
763,okaihau,NZ,286.21,scattered clouds,-35.3167,173.7833,Macadamia Farm
765,mezen,RU,286.20,broken clouds,65.8522,44.2400,Gostevoy Dom Everest


In [58]:
# 8a. Create the output File (CSV)
output_data_file = "weather_data/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [59]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>city</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [60]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))